# Packages

In [1]:
from quocslib.optimalcontrolproblems.OneQubitProblem_2fields import OneQubit2Fields
import time, datetime

import scipy
import numpy as np
from scipy import linalg
import os
import sys
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.optimize import minimize
import time
import copy as cp
from scipy import signal
# import necessary stuff from quidi
from logic.pulsed.pulse_objects import PulseBlock, PulseBlockEnsemble, PulseSequence
from user_scripts.Timo.own.optimalcontrol.oc_simlib import TimeDependentSimulation, SimParameters, ArbPulse, PredefinedArbPulses

import logging
logging.basicConfig(filename='logfile.log', filemode='w', level=logging.DEBUG)
logger = logging.getLogger(__name__)

0

0

# Library functions

In [2]:

def list_2_csv(in_list):
    str_list = ""
    
    if type(in_list) != list:
        in_list = [in_list]
        
    for el in in_list:
        str_list += f"{el}, "
    
    if len(str_list) > 0:
        str_list = str_list[:-2]
        
    return str_list

def dict_2_header_str(in_dict):
    out_str = ""
    for key, val in in_dict.items():
        out_str += f"{key}: {val}\n"
        
    return out_str

def get_pulse_filename(path, name="", name_ampl="amplitude.txt", name_phase="phase.txt"):
    return os.path.abspath(path + "/" + name + name_ampl), os.path.abspath(path + "/" + name + name_phase)

def save_pulse(path, data_ampl, data_phase, name="", name_ampl="amplitude.txt", name_phase="phase.txt"):
    header_dict = {}
    try:
        now = datetime.datetime.now() # current date and time
        header_dict['timestamp'] = now.strftime("%Y/%m/%d-%H:%M:%S")
        header_dict['script_parameters'] = script_params
        header_dict.update(opti_comm_dict['optimization_dictionary'])
        
        fname_ampl, fname_phase = get_pulse_filename(path, name=name, name_ampl=name_ampl, name_phase=name_phase)
        
        if data_ampl is not None:
            np.savetxt(fname_ampl,  data_ampl,  header=dict_2_header_str(header_dict))
        if data_phase is not None:
            np.savetxt(fname_phase, data_phase, header=dict_2_header_str(header_dict))
    except:
        logger.exception("")

        
def save_optimization_result(path, name="opt_res_", name_ampl="amplitude.txt", name_phase="phase.txt"):
    try:
        # save fom by same function as pulse
        save_pulse(path, fom_all, None, name=name, name_ampl='fom.txt', name_phase='')
        
        optimizer_obj = optimizationlogic.optimization_obj
        best_dict = optimizer_obj.opt_alg_obj.get_best_controls()
        
        pulses_list = best_dict['pulses']
        time_grids_list = best_dict['timegrids']
        parameters_list = best_dict['parameters']
        
        t_amplitude = time_grids_list[0]
        amplitude = pulses_list[0]
        t_phase = time_grids_list[1]
        phase = pulses_list[1]

        data_ampl = np.column_stack((t_amplitude, amplitude))
        data_phase = np.column_stack((t_phase, phase))
        fom = np.column_stack((range(len(fom_all)), fom_all))

        save_pulse(path, data_ampl, data_phase, name=name, name_ampl=name_ampl, name_phase=name_phase)

    except:
        logger.exception("")
        
#save_pulse(folder_path, np.asarray([1,2]), np.asarray(1))
#get_pulse_filename(folder_path, name=filename)
0

0

# QuOCS and Qudi with Noise

## Creation of the optimization dictionary

The optimization dictionary contains all the settings compulsory for the optimization algorithm in order to run a proper optimization.

In [3]:
optimization_client_name = "test_dCRAB_Noisy_2_control_fields"

optimization_dictionary = {"optimization_client_name": optimization_client_name,
                           'opti_algorithm_module': 'quocslib.optimalalgorithms.dCRABNoisyAlgorithm', 
                           'opti_algorithm_class': 'DCrabNoisyAlgorithm', 
                          }

Number of iteration and super-iterations

In [4]:
# Total number of dCRAB superiteration
super_iteration_number = 10000000
# Maximum number of iteration per super-iteration (in max_eval_total: maximale anzahl von evaluation steps gesamt)
maximum_function_evaluations_number = 500

To activate the drift compensation and the re-evaluation steps add

In [5]:
optimization_dictionary['algorithm_settings'] = {"algorithm_name": "dCRAB",
                                                 "super_iteration_number": super_iteration_number,
                                                 #"max_eval_total": super_iteration_number * maximum_function_evaluations_number,
                                                 "FoM_goal": 0.00001,
                                                 "total_time_lim": 2*48*60,  # minutes
                                                 "compensate_drift": {
                                                     "compensate_after_SI": True,
                                                     "compensate_after_minutes": 15
                                                 },
                                                 "random_number_generator":{
                                                     "seed_number":420   # todo: really fix a seed here?
                                                 },
                                                 #"re_evaluation": {
                                                     #"re_evaluation_steps": [0.3, 0.5, 0.51]
                                                 #}
}
optimization_dictionary['algorithm_settings'] 

{'algorithm_name': 'dCRAB', 'super_iteration_number': 10000000, 'FoM_goal': 1e-05, 'total_time_lim': 5760, 'compensate_drift': {'compensate_after_SI': True, 'compensate_after_minutes': 15}, 'random_number_generator': {'seed_number': 420}}

Settings for the inner algorithm used by dCRAB

In [6]:
optimization_dictionary['algorithm_settings']["dsm_settings"] = {'general_settings': {
                                                "dsm_algorithm_name": "NelderMead", 
                                                'is_adaptive': False
                                            }, 
                                            'stopping_criteria': {
                                                #"max_eval": 100,
                                                "time_lim": 2*90,  #min
                                                #"xatol": 1e-10, 
                                                #"frtol": 1e-10,
                                                "change_based_stop": {
                                                    "cbs_funct_evals": 500, 
                                                    "cbs_change": 0.01    # stop if slope below
                                                }
                                            }
}
optimization_dictionary['algorithm_settings']

{'algorithm_name': 'dCRAB', 'super_iteration_number': 10000000, 'FoM_goal': 1e-05, 'total_time_lim': 5760, 'compensate_drift': {'compensate_after_SI': True, 'compensate_after_minutes': 15}, 'random_number_generator': {'seed_number': 420}, 'dsm_settings': {'general_settings': {'dsm_algorithm_name': 'NelderMead', 'is_adaptive': False}, 'stopping_criteria': {'time_lim': 180, 'change_based_stop': {'cbs_funct_evals': 500, 'cbs_change': 0.01}}}}

### Times

In [7]:
t_rabi_rect = 76.4e-9  
pix_pulse = 1
t_oc_pulse = 100e-9 #1.0000000000000001e-07
n_timebins = 2000
#t_oc_pulse = timegrid_ampl[-1]
fac_oc_guess = 1.0 # make initial guess not too good, otherwise hard for optimizer

ampl_max = 0.25  # V
ampl_rect = 0.25  # V 


fac_oc_rect = t_oc_pulse/(0.5*t_rabi_rect/pix_pulse)
if fac_oc_rect  < 3:
    logger.warning(f"OC pulse should be >3x as long as rect, now: {t_oc_pulse/(0.5*t_rabi_rect/pix_pulse)}")

time_p = {'time_name': 'time_p', 'initial_value': t_oc_pulse} # pulse length
optimization_dictionary['times'] = [time_p]
ampl_oc_guess = fac_oc_guess*1/fac_oc_rect*ampl_rect

ampl_oc_guess, fac_oc_rect
t_oc_pulse, n_timebins

(1e-07, 2000)

### Parameters

In [8]:
optimization_dictionary['parameters'] = []

In [9]:
optimization_dictionary['times']

[{'time_name': 'time_p', 'initial_value': 1e-07}]

### Pulses

### Generate from predefined function

Levitt composite pulse

In [100]:
pulse = PredefinedArbPulses.generate_levitt(20e6, n_t=1000)
pulse._func_omega_mhz_2_ampl_v = lambda x: x/20*0.25

from scipy.ndimage import gaussian_filter1d
pulse._data_ampl = gaussian_filter1d(pulse._data_ampl, 5)
pulse._data_phase = gaussian_filter1d(pulse._data_phase, 5)

#pulse.as_dict()

Rect pulse

In [164]:
def generate_rect_pi(omega, phase=0, n_t=1000, t_pulse=None):
    """
    Generate a levitt pulse as a optimal control pulse file.
    Assumes that quadratues I*sin(f_mw*t) + Q*cos(f_mw*t) are used in sampling.
    @param t_pulse: if None, auto length of pulse. Else zero pad to specified length.
    """

    omega_mhz = omega * 1e-6

    get_t_pix = PredefinedArbPulses.get_t_pix
    get_iq = PredefinedArbPulses.get_iq

    tpi_us = get_t_pix(omega_mhz, pix=1)

    if not t_pulse:
        t_pulse = tpi_us
    else: 
        t_pulse = 1e6*t_pulse

    # rabi in MHz, times in us
    timegrid_us = np.linspace(0, t_pulse, n_t)
    data_ampl = np.zeros((len(timegrid_us)))  # I quadrature
    data_phase = np.zeros((len(timegrid_us)))  # Q

    phases = [0 + phase]
    tpulse_by_pi = [1]
    # pad with zero amplitude/phase
    if t_pulse != tpi_us:
        phases.append(0)
        tpulse_by_pi.append(0)
    # phases = [np.pi/2]
    # tpulse_by_pi = [0.5]

    t_curr_us, t_end_us = 0, 0
    for i_comppulse, phi in enumerate(phases):
        pix = tpulse_by_pi[i_comppulse]
        if pix != 0:
            t_end_us = t_curr_us + get_t_pix(omega_mhz, pix=pix)
            val_iq = np.asarray(get_iq(phi)) * omega_mhz
        else:  # zero pad until end
            t_end_us = timegrid_us[-1]
            val_iq = np.asarray([0, 0])

        idx_start = np.argmin(np.abs(timegrid_us - t_curr_us))
        idx_end = np.argmin(np.abs(timegrid_us - t_end_us))


        data_ampl[idx_start:idx_end + 1] = val_iq[0]
        data_phase[idx_start:idx_end + 1] = val_iq[1]

        if t_curr_us == timegrid_us[-1]:
            break
        t_curr_us = t_end_us

    idx_end = np.argmin(np.abs(timegrid_us - t_end_us))

    assert idx_end == n_t - 1

    pulse = ArbPulse()
    pulse.name = 'rect_phi={phase/np.pi:.1f}pi'

    pulse.timegrid_unit = 'us'
    pulse.data_unit = 'MHz'

    pulse._data_ampl = data_ampl
    pulse._data_phase = data_phase
    pulse._timegrid_ampl = timegrid_us
    pulse._timegrid_phase = timegrid_us

    return pulse

PredefinedArbPulses.generate_rect_pi = generate_rect_pi    

In [190]:
t_oc_pulse

1e-07

In [188]:
np.linspace(0,t_oc_pulse,10)[-5:]*1e6

array([0.05555556, 0.06666667, 0.07777778, 0.08888889, 0.1       ])

In [172]:
1e3*np.linspace(0,t_oc_pulse)[-1]

9.999999999999999e-05

In [200]:
# pi pulse with fixed lentgh
pulse = PredefinedArbPulses.generate_rect_pi(20e6, n_t=n_timebins, t_pulse=t_oc_pulse)
# hard pi pulse with auto lentgh
#pulse = PredefinedArbPulses.generate_rect_pi(20e6, n_t=1000)
pulse._func_omega_mhz_2_ampl_v = lambda x: x/20*0.25

fname = 'dummy' # to make loadable below

pulse.as_dict()
#pulse._timegrid_phase[-1]


{'name': 'rect_phi={phase/np.pi:.1f}pi', 'folder': None, 'file': [], 'timegrid_unit': 'us', 'data_unit': 'MHz', 'data_ampl': array([20., 20., 20., ...,  0.,  0.,  0.]), 'data_phase': array([0., 0., 0., ..., 0., 0., 0.]), 'timegrid_ampl': array([0.00000000e+00, 5.00250125e-05, 1.00050025e-04, ...,
       9.98999500e-02, 9.99499750e-02, 1.00000000e-01]), 'timegrid_phase': array([0.00000000e+00, 5.00250125e-05, 1.00050025e-04, ...,
       9.98999500e-02, 9.99499750e-02, 1.00000000e-01])}

#### Load initial guess from extern
... or with fname=None assume slow rect pulse.

In [12]:
#folder = r"C:\Software\qudi_data\optimal_control_assets"
folder = r"C:\Software\qudi_data\optimal_control_assets\20220725_2_lessClipping"
folder = r"C:\Software\qudi_data\optimal_control_assets\20220726_1_NV1_penalty_fac=1"

#fname =  r"on_nv=2_"
fname = r"opt_res_"

folder = r"C:\Software\qudi_data\optimal_control_assets\Reisser_MSc_pulse_1"
fname = r"Opt_Pulse_01_"

folder = r"C:\Software\qudi_data\optimal_control_assets\20220726_1_NV1_penalty_fac=1"
folder = r"C:\Software\qudi_data\optimal_control_assets\Reisser_MSc_pulse_1"
folder =  r"C:\Software\qudi_data\optimal_control_assets\20220727_1_guessFromReisserMsc"
folder = r"C:\Software\qudi_data\optimal_control_assets\20220817_1_guessFromReisserMsc_penalty_fac=100"
#fname =  r"on_nv=2_"
fname = r"opt_res_"
#fname = r"Opt_Pulse_01_"




#pulse = ArbPulse.load_pulse(folder, fname, unit_t='s', unit_data='V')

pulse = ArbPulse.load_pulse(folder, fname, unit_t='s', unit_data='V',
                            func_ampl_v_2_omega_mhz=lambda x: x/0.25*20,
                            func_omega_mhz_2_ampl_v=lambda x: 1/(x/0.25*20))

# rescale amplitude if needed
# NOW rabi/ BACK rabi
pulse._data_ampl  = 76.4/49.3*pulse._data_ampl
pulse._data_phase = 76.4/49.3*pulse._data_phase

# ONLY for loading Reisser pulse
#pulse = ArbPulse.load_pulse(folder, fname, unit_t='ns', unit_data='MHz',
#                            func_omega_mhz_2_ampl_v=lambda y: y/13.849* 0.25 * 13.849/20)

np.max(pulse._data_ampl), np.max(pulse._data_phase), pulse.as_dict()

(0.29954136343199284, 0.22210290736814414, {'name': 'opt_res_', 'folder': 'C:\\Software\\qudi_data\\optimal_control_assets\\20220817_1_guessFromReisserMsc_penalty_fac=100', 'file': ['C:\\Software\\qudi_data\\optimal_control_assets\\20220817_1_guessFromReisserMsc_penalty_fac=100\\opt_res_amplitude.txt', 'C:\\Software\\qudi_data\\optimal_control_assets\\20220817_1_guessFromReisserMsc_penalty_fac=100\\opt_res_phase.txt'], 'timegrid_unit': 's', 'data_unit': 'V', 'data_ampl': array([0.11121797, 0.10998275, 0.10876691, ..., 0.01157454, 0.01086255,
       0.01014398]), 'data_phase': array([ 0.01565203,  0.01625656,  0.01685176, ..., -0.00390291,
       -0.00451652, -0.00513233]), 'timegrid_ampl': array([0.00000000e+00, 5.00250125e-11, 1.00050025e-10, ...,
       9.98999500e-08, 9.99499750e-08, 1.00000000e-07]), 'timegrid_phase': array([0.00000000e+00, 5.00250125e-11, 1.00050025e-10, ...,
       9.98999500e-08, 9.99499750e-08, 1.00000000e-07])})

In [11]:
pulse.data_ampl
folder, fname

('C:\\Software\\qudi_data\\optimal_control_assets\\20220817_1_guessFromReisserMsc_penalty_fac=100', 'opt_res_')

### Set initial guess

In [13]:
#fname=None

if fname:
    try:        
        pulse = pulse
        timegrid_ampl = pulse.get_timegrid(unit='s')
        len_oc_pulse = optimization_dictionary['times'][0]['initial_value']
        if n_timebins != len(timegrid_ampl) or not np.isclose(len_oc_pulse,timegrid_ampl[-1]):
            raise NotImplementedError(f"Timegrid of loaded init pulse ({timegrid_ampl[-1]}@{len(timegrid_ampl)})\
                                      doesn't match settings ({len_oc_pulse}@{n_timebins}). Consider implementing interpolation!")
    except:
        logger.exception("")
        fname = None
if fname:
    try:
        initial_guess_ampl = {'function_type': 'list_function', 'list_function': pulse.get_data_ampl(unit='V')}
        initial_guess_phase = {'function_type': 'list_function', 'list_function': pulse.get_data_phase(unit='V')}
    except:
        logger.exception("")
    # ATTENTION: ONLY FOR DEBUG
    #initial_guess_ampl = {'function_type': 'list_function', 'list_function': 1/1.4*pulse['data_ampl']}
    #initial_guess_phase = {'function_type': 'list_function', 'list_function': 1/1.4*pulse['data_phase']}
else:
    initial_guess_ampl = {'function_type': 'lambda_function', 'lambda_function': f'lambda t: {ampl_oc_guess} + 0.0*t'}
    initial_guess_phase = {'function_type': 'lambda_function', 'lambda_function': f'lambda t: {ampl_oc_guess} + 0.0*t'}
    
initial_guess_ampl['function_type']

'list_function'

In [14]:
try:
    pulse.get_data_ampl(unit='V')
except:
    logger.exception("")

In [15]:
np.max(pulse.get_data_ampl(unit='V'))

0.29954136343199284

In [16]:
vec_number = 3   #2-3
upper_freq_lim = 12  #8-12


    
pulse_amplitude = {'pulse_name': 'Amplitude', 
                           'upper_limit':ampl_max, 
                           'lower_limit': -ampl_max, 
                           'bins_number': n_timebins, 
                           'time_name': 'time_p', 
                           'amplitude_variation': 0.5*ampl_oc_guess, # added to guess amp, to create 3 points for start simplex
                           'basis': {'basis_name': 'Fourier', 
                                     'basis_class': 'Fourier', 
                                     'basis_module': 'quocslib.pulses.basis.Fourier', 
                                     'basis_vector_number': vec_number, # number of frequencies within the below defined range
                                     'random_super_parameter_distribution': 
                                     {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                                           'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                                           'lower_limit': 0.0, 'upper_limit': upper_freq_lim} # number of oscillations within pulse
                                    }, 
                           'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
                           'initial_guess': initial_guess_ampl,
                           'shrink_ampl_lim': True
                          }

pulse_phase = {'pulse_name': 'Phase', 
               'upper_limit': ampl_max, 
               'lower_limit': -ampl_max, 
               'bins_number': n_timebins, 
               'time_name': 'time_p', 
               'amplitude_variation': 0.5*ampl_oc_guess, # added to guess amp, to create 3 points for start simplex
               'basis': {'basis_name': 'Fourier', 
                         'basis_class': 'Fourier', 
                         'basis_module': 'quocslib.pulses.basis.Fourier', 
                         'basis_vector_number': vec_number, # number of frequencies within the below defined range
                         'random_super_parameter_distribution': 
                         {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                               'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                               'lower_limit': 0.0, 'upper_limit': upper_freq_lim} # number of oscillations within pulse
                        }, 
               'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
               'initial_guess': initial_guess_phase,
               'shrink_ampl_lim': True
              }

optimization_dictionary['pulses'] = [pulse_amplitude, pulse_phase]

ampl_oc_guess

0.0955

In [125]:

optimization_dictionary['pulses'][0]['initial_guess']

{'function_type': 'list_function', 'list_function': array([0.07875513, 0.07788045, 0.0770195 , ..., 0.00819611, 0.00769194,
       0.00718311])}

### Init & Transfer to Qudi-QuOCS

In [17]:
opti_comm_dict = {"optimization_dictionary": optimization_dictionary}
optimizationlogic.load_opti_comm_dict(opti_comm_dict)

Press "Start" in the gui and see whether errors are logged.

Print the optimization dictionary also here. If the GUI is not showing the optimization dictionary, restart the Kernel.

In [18]:
optimalcontrol.opti_comm_dict

{'optimization_dictionary': {'optimization_client_name': 'test_dCRAB_Noisy_2_control_fields', 'opti_algorithm_module': 'quocslib.optimalalgorithms.dCRABNoisyAlgorithm', 'opti_algorithm_class': 'DCrabNoisyAlgorithm', 'algorithm_settings': {'algorithm_name': 'dCRAB', 'super_iteration_number': 10000000, 'FoM_goal': 1e-05, 'total_time_lim': 5760, 'compensate_drift': {'compensate_after_SI': True, 'compensate_after_minutes': 15}, 'random_number_generator': {'seed_number': 420}, 'dsm_settings': {'general_settings': {'dsm_algorithm_name': 'NelderMead', 'is_adaptive': False}, 'stopping_criteria': {'time_lim': 180, 'change_based_stop': {'cbs_funct_evals': 500, 'cbs_change': 0.01}}}}, 'times': [{'time_name': 'time_p', 'initial_value': 1e-07}], 'parameters': [], 'pulses': [{'pulse_name': 'Amplitude', 'upper_limit': 0.25, 'lower_limit': -0.25, 'bins_number': 2000, 'time_name': 'time_p', 'amplitude_variation': 0.04775, 'basis': {'basis_name': 'Fourier', 'basis_class': 'Fourier', 'basis_module': 'quo

# Measurement

Run a normal rect Rabi with all analysis and MW generation parameters before. Then run with commented out the dicts below in order to keep all the respective pulsed settings.

In [69]:
script_params = {}
script_params['runtime_per_epoch'] = 40  # s
script_params['readout_noise'] = np.sqrt(2)*0.01  # std_dev of fom, controls convergence in iteration
# needs to be smaller than the actual pulse length time_p due to the interpolation
script_params['length_oc'] = t_oc_pulse - 1e-9
script_params['is_noisy'] = True

# file paths 
folder_path = r'C:\Software\qudi_data\optimal_control_assets'
filename= "on_nv=1_"

script_params['readout_noise']

0.014142135623730952

## Set measurement sequence and fom

In [70]:
def setup_sts_oc_pi(folder_path, filename, length_oc):
    dictparamsall = pulsedmasterlogic.generate_method_params
    
    dict_qst = dictparamsall["sts_oc"]
    dict_qst["experiment"] = 'sts_oc'
    dict_qst["name"] = 'stsoc'
    dict_qst["length"] = length_oc
    dict_qst["filename_amplitude"] = os.path.basename(get_pulse_filename(folder_path, name=filename)[0])
    dict_qst["filename_phase"] = os.path.basename(get_pulse_filename(folder_path, name=filename)[1])
    dict_qst["folder_path"] = folder_path
    

    
    return dict_qst

def setup_oc_npi(folder_path, filename, length_oc, n_start=1, n_step=1, n_n=4, t_gap=0e-9, phases_oc=[0],
                init_end_pix=0, init_end_phases_deg=[0]):
    dictparamsall = pulsedmasterlogic.generate_method_params
    
    dict_qst = dictparamsall["oc_nrep"]
    dict_qst["experiment"] = 'oc_nrep'
    dict_qst["name"] = 'oc_nrep'
    dict_qst["filename_amplitude"] = os.path.basename(get_pulse_filename(folder_path, name=filename)[0])
    dict_qst["filename_phase"] = os.path.basename(get_pulse_filename(folder_path, name=filename)[1])
    dict_qst["folder_path"] = folder_path
    
    dict_qst['n_start'] = n_start
    dict_qst['n_step'] = n_step
    dict_qst['t_gap'] = t_gap
    dict_qst['phases'] = list_2_csv(phases_oc)
    dict_qst['init_end_pix'] = init_end_pix
    dict_qst['init_end_phases_deg'] = list_2_csv(init_end_phases_deg)
    dict_qst['num_of_points'] = n_n
    
    dict_qst["alternating"] = True
    dict_qst["vs_rect_pulse"] = False
    
    
    return dict_qst

def setup_fom_func_sts_oc_pi():
    def calc_fom():
        signal_data = pulsedmeasurementlogic.signal_data
        up_norm = signal_data[1,0] / signal_data[1,0]
        down_norm = signal_data[1,1] / signal_data[1,0]

        return np.real(1 - (up_norm - down_norm))
    
    return calc_fom

def setup_fom_func_oc_npi():
    def calc_fom():
        # signal_data[0,:] = xaxis
        # signal_data[1,:] = yaxis (alternating flattened)
        try:
            signal_data = pulsedmeasurementlogic.signal_data
            n_pi = signal_data[0,:]
            n_pi_even = np.where(n_pi%2==0)

            data_mw = signal_data[1,:]
            data_alt = signal_data[2,:]
            data_norm = data_mw/data_alt
            data_mw_0 = (data_norm)[n_pi%2==0]  # normed in state |0>
            data_mw_1 = (data_norm)[n_pi%2==1]  # normed in state |1>

            return np.mean(1 - (data_mw_0 - data_mw_1))
        except:
            return np.nan
    
    return calc_fom

def setup_fom_simulation_sigz(f_res):
    
    def fom_func(pulse, func_volt_2_rabi=None):
        # sim units are MHz!
        simp = SimParameters()

        B_gauss = (simp.D - f_res*1e-6)/ simp.gamma_nv

        # reloading pulse to make sure units are correct
        pulse = ArbPulse.load_pulse(pulse._folder, pulse.name, extension='txt',
                                     unit_t='s', unit_data='V', 
                                      func_ampl_v_2_omega_mhz=func_volt_2_rabi
                                      )

        try:
            sim = TimeDependentSimulation()

            freq_array = np.asarray([f_res])/1e6  # MHz
            logger.debug(f"{freq_array}")
            data_freq_detuning = sim.run_sim_fsweep(freq_array, pulse, B_gauss, simp)
            spin_pop = np.mean(data_freq_detuning)
            return spin_pop

        except Exception as e:
            logger.exception(f"Couldn't simulate pulse response: ")
            return np.nan
            
        
    
    return fom_func
    
    


def add_penalty_freq_bandstop(fom_func, f_res, f_center_block, df=10e6, n_f_sim=50, fac_penalty=1, symm_block=True):
    def get_f_sim_array(f_res, f_center_block, df, n_f, symm_block=False):
        f_center_block_mhz = f_center_block*1e-6
        f_res_mhz = f_res*1e-6
        df_mhz = df*1e-6
        if symm_block:
            delta_block = f_center_block_mhz - f_res_mhz
            f_center_2_mhz = f_res_mhz - delta_block
            f_centers = [f_center_block_mhz, f_center_2_mhz]
        else:
            f_centers = [f_center_block_mhz]

        freqs = []
        for f in f_centers:
            freqs.extend(list(np.linspace(f-df_mhz, f+df_mhz, int(n_f/len(f_centers)))))
        freq_array = np.asarray(freqs)

        return freq_array

    def extend_fom(pulse, func_volt_2_rabi=None):
        exp_fom = fom_func()

        # sim units are MHz!
        simp = SimParameters()


        B_gauss = (simp.D - f_res*1e-6)/ simp.gamma_nv

        # reloading pulse to make sure units are correct
        pulse = ArbPulse.load_pulse(pulse._folder, pulse.name, extension='txt',
                                     unit_t='s', unit_data='V', 
                                      func_ampl_v_2_omega_mhz=func_volt_2_rabi
                                      )

        try:
            sim = TimeDependentSimulation()
            
            freq_array = get_f_sim_array(f_res, f_center_block, df, n_f_sim, symm_block=symm_block)
            data_freq_detuning = sim.run_sim_fsweep(freq_array, pulse, B_gauss, simp)
            spin_pop = np.mean(data_freq_detuning)


        except Exception as e:
            logger.exception(f"Couldn't simulate pulse response: ")
            return np.nan

        # in the blockband, spin_pop should be uneffected (=1)
        fom_sim = (1 + fac_penalty*(abs(1-spin_pop)))
        fom_hybrid = exp_fom * fom_sim
        logger.debug(f"FOM: {fom_hybrid} <= Exp: {exp_fom}, Sim: {fom_sim},  factor= {fac_penalty}")
        logger.debug(f"Block bands: {freq_array}, nf= {len(freq_array)}")
        return fom_hybrid


    return extend_fom

script_params['fom_func_volt_2_rabi'] = None
qmeas = setup_sts_oc_pi(folder_path, filename, script_params['length_oc'])
qmeas = setup_oc_npi(folder_path, filename, script_params['length_oc'])
# for OC pulse on NV2 (control): from states 0, 1
qmeas = setup_oc_npi(folder_path, filename, script_params['length_oc'], t_gap=0.25/2.1e6, n_n=4)
qmeas = setup_oc_npi(folder_path, filename, script_params['length_oc'], t_gap=1/2.1e6, n_n=8)
qmeas = setup_oc_npi(folder_path, filename, script_params['length_oc'], t_gap=0, n_n=4)
#qmeas = setup_oc_npi(folder_path, filename, script_params['length_oc'], t_gap=0, n_n=8)

# for OC pulse on NV1 (target): from states X,Y
# todo: IS IT ENOUGH to do pi2X-MWX-pi2X and pi2Y-MWY-pi2Y or do we need pi2X-MWY-pi2X (spin lock)
#qmeas = setup_oc_npi(folder_path, filename, script_params['length_oc'], t_gap=0.25/2.1e6, n_n=4,
#                    phases_oc=[0, 90], init_end_pix=0.5, init_end_phases_deg=[0, 90])
# From simulated ProcTomo: OC pi from ResserMsc is roughly Y pulse
# => need to adapt phases
#qmeas = setup_oc_npi(folder_path, filename, script_params['length_oc'], n_n=4, t_gap=2/2*1/2.1e6, 
#                    #phases_oc=[0,90], init_end_pix=0.5, init_end_phases_deg=[0, 90])
#                     phases_oc=[0], init_end_pix=0.5, init_end_phases_deg=[0])




f_res = 2609.56e6
f_block = 2718e6
fom_func = setup_fom_func_oc_npi()


script_params['fom_func_volt_2_rabi'] = lambda y: y/ampl_rect*(1e-6/t_rabi_rect)
fom_func = add_penalty_freq_bandstop(fom_func, f_res, f_block, fac_penalty=10, symm_block=True)
#fom_func = add_penalty_freq_bandstop(fom_func, f_res, f_block, fac_penalty=1, symm_block=True)

script_params['generate_parameters'] = qmeas
script_params['fom_function'] = fom_func

script_params['generate_parameters']



{'name': 'oc_nrep', 'n_start': 1, 'n_step': 1, 'num_of_points': 4, 'filename_amplitude': 'on_nv=1_amplitude.txt', 'filename_phase': 'on_nv=1_phase.txt', 'folder_path': 'C:\\Software\\qudi_data\\optimal_control_assets', 't_gap': 0, 'phases': '0', 'init_end_pix': 0, 'init_end_phases_deg': '0', 'vs_rect_pulse': False, 'alternating': True, 'experiment': 'oc_nrep'}

In [23]:
1/2*1/2.1e6  *1e6

0.23809523809523808

In [54]:
# debug fom
try:
    data = script_params['fom_function'](pulse, func_volt_2_rabi=lambda y: y/0.25*20)
except:
    logger.exception("")
    
data

0.9137020096219715

In [243]:
#pulse = #folder = r"C:\Software\qudi_data\optimal_control_assets"
folder = r"C:\Software\qudi_data\optimal_control_assets\20220724_1"
fname = r"on_nv=1_"
#fname = r"opt_pulse_"

# Change from TimeDependentsSimulation to ArbPulse
#pulse = TimeDependentSimulation.load_pulse(folder, fname, extension='txt',
                                           #func_volt_2_rabi=func_volt_2_rabi,
 #                                          func_t_2_us=lambda x: x*1e6)
pulse = ArbPulse.load_pulse(folder, fname, extension='txt',
                                           #func_volt_2_rabi=func_volt_2_rabi,
                                           func_t_2_us=lambda x: x*1e6)
pulse['timegrid_ampl'][-1]

Traceback (most recent call last):
  File "C:\Software\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-243-fdaf027881ee>", line 12, in <module>
    func_t_2_us=lambda x: x*1e6)
TypeError: load_pulse() got an unexpected keyword argument 'func_t_2_us'


In [ ]:
signal_data = pulsedmeasurementlogic.signal_data
n_pi = signal_data[0,:]
data_mw = signal_data[1,:]
data_alt = signal_data[2,:]
data_norm = data_mw/data_alt
data_mw_0 = (data_norm)[n_pi%2==0]
data_mw_1 = (data_norm)[n_pi%2==1]

data_mw_1

## Run

In [68]:

time.time()
def set_optimize_time(qmeas, t_last_opt):
    """
    override the default tracking/optimize behavior:
    optimize once at startup of OC optimization step if optimize time is reached
    """
    qmeas = cp.copy(qmeas)
    if 'optimize_time' in qmeas.keys():
        if time.time() - t_last_opt > qmeas['optimize_time']:
            # optimize after mes start
            qmeas['optimize_time'] = 1
            t_last_opt = time.time()
        else:
            qmeas.pop('optimize_time')
    return qmeas, t_last_opt



#set_optimize_time(qmeas, 0)
# todo:buggy

In [ ]:
######################################################################################################
# Parameters and Settings
######################################################################################################

# runtime of each experiment
runtime = script_params['runtime_per_epoch']
length_oc = script_params['length_oc'] 

# parameter to stop the experiment if its set to False in the console
pulsedmasterlogic.globalrun = True
uglobals.abort.clear()
logger.info("Starting new closed-loop quocs optimization")

# Dictonary containing the general measurement parameters for the predefined measurements
p_generation_dict = pulsedmasterlogic.generation_parameters
#p_generation_dict["laser_channel"] = "d_ch2" 
#p_generation_dict["sync_channel"] = "d_ch1" 
#p_generation_dict["gate_channel"] = None#"d_ch1" 
#p_generation_dict["microwave_channel"] = "a_ch1"
#p_generation_dict["microwave_amplitude"] = 0.25
#p_generation_dict["microwave_frequency"] = 2822.3e6
#p_generation_dict["rabi_period"] = 82.7e-9
#p_generation_dict["laser_length"] = 3e-06
#p_generation_dict["laser_delay"] = 0.2e-6
#p_generation_dict["wait_time"] = 1e-06
#p_generation_dict["analog_trigger_voltage"] = 0.0

# load the general measurement parameters for the predefined measurements
pulsedmasterlogic.set_generation_parameters(p_generation_dict)
script_params['pulsed_generation_params'] = pulsedmasterlogic.generation_parameters


# tell the measurement gui how the sequence looks
pulsed_settings = dict()
pulsed_settings['invoke_settings'] = True
pulsedmasterlogic.set_measurement_settings(pulsed_settings)
script_params['pulsed_settings'] = pulsed_settings


# make sure everything is finished
time.sleep(5)

# how the laser response is analyzed    
#pulsedmasterlogic.set_analysis_settings(method = 'mean')
#pulsedmasterlogic.set_analysis_settings(signal_start = 150e-9)
#pulsedmasterlogic.set_analysis_settings(signal_end = 1e-6)
#pulsedmasterlogic.set_extraction_settings(method = 'conv_deriv')

p_analysis_settings = pulsedmasterlogic.analysis_settings
p_extraction_settings = pulsedmasterlogic.extraction_settings
script_params['pulsed_analysis_settings'] = p_analysis_settings
script_params['pulsed_extraction_settings_settings'] = p_extraction_settings

# array to save the fom evolution
fom_all = []

# This section is devoted to the initialization in the pulsed logic and optimization logic of the main
# settings and parameters to be usde in the creation ofthe pulse sequence and the optimization
# Iteration, controls and figure of merit to compare with QuOCS
# Just an example for debug
args_dict = {"is_noisy": script_params['is_noisy']}
qubit = OneQubit2Fields(args_dict)

######################################################################################################
# Measurement
######################################################################################################
optimalcontrol.start_optimization()

# crucial, maybe it can be much shorter
time.sleep(1)

# Just a time to check for latent time
last_time_fom = time.time()
# repeat the whole process until its manually stopped or QuOCS finsihed the optimization
# Wait few seconds before starting to get and return data
while not optimizationlogic.handle_exit_obj.is_user_running:
    time.sleep(0.1)
    if (time.time() - last_time_fom) > 30 or uglobals.abort.is_set():
        logger.warning("Timeout or abort while initializing quocs.")
        uglobals.abort.set()
        break

# iteration number
it_val = 0

# when did the optimization start?
opt_start_time = time.time()
t_last_optimize = 0
try:
    # print("Check before the loop starts: {0}".format(optimizationlogic.handle_exit_obj.is_user_running))
    while optimizationlogic.handle_exit_obj.is_user_running == True and not uglobals.abort.is_set():
        time_stamp=time.time()
        # wait until QuOCS optimizes the controls
        # print("Wait until the controls logic gives the controls")
        while not controlslogic.are_pulses_calculated:
            time.sleep(0.1)
            # If the waiting time exceed 10 seconds left stop the optimization
            if time.time() - last_time_fom > 20:
                logger.error("Too much time... Exit!")
                optimizationlogic.handle_exit_obj.is_user_running = False
                break

 
        #######################################################################################################
        # Get the Controls
        #######################################################################################################
        # Change the status of control calculations to avoid to evaluate the fom twice with the same controls
        controlslogic.are_pulses_calculated = False
        # Get the controls from the controls logic
        pulses, parameters, timegrids = controlslogic.pulses, controlslogic.parameters, controlslogic.timegrids
        #######################################################################################################
        # Perform the measurement
        #######################################################################################################

        # save the pulses as .txt files (predefined methods doesn't allow us to upload a numpy array as 
        # parameter)
        data_ampl = np.column_stack((timegrids[0], pulses[0]))
        data_phase = np.column_stack((timegrids[0], pulses[1]))
        save_pulse(folder_path, data_ampl, data_phase, name=filename)

        time.sleep(0.5)
        
        qmeas = cp.deepcopy(script_params['generate_parameters'])
        experiment = qmeas['experiment']
        exp_name = qmeas['name']
        qmeas.pop('experiment', None)
        
        #qmeas = set_optimize_time(qmeas, t_last_optimize)
        # generate the sequence
        pulsedmasterlogic.generate_predefined_sequence(experiment,kwarg_dict=qmeas)  

        time.sleep(1)

        # upload the sequence
        pulsedmasterlogic.sample_ensemble(exp_name,True)
        while pulsedmasterlogic.status_dict['sampload_busy'] or pulsedmasterlogic.status_dict['sampling_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:     
            time.sleep(1)

        #print('Finished uploading', dictparams_oc["name"], '!')

        # make sure everything is finished (crucial)
        #time.sleep(1)#time.sleep(5)    

         # start the measurement
        #print('Starting the measurement!')
        pulsedmasterlogic.toggle_pulsed_measurement(True)
        time.sleep(1)

        # make sure the measurement started
        while not pulsedmasterlogic.status_dict['measurement_running']:
            time.sleep(0.1)

        measurement_start_time = time.time()

        while time.time() <= measurement_start_time + runtime:
            time.sleep(0.2)

            # option to stop the measurement
            if pulsedmasterlogic.globalrun == False or uglobals.abort.is_set():
                print('Stopping the measurement!')
                break

        # option to stop the measurement
        if pulsedmasterlogic.globalrun == False or uglobals.abort.is_set():
            print('Stopping the measurement!')
            pulsedmasterlogic.toggle_pulsed_measurement(False)
            break

        #time.sleep(1)

        # Stop the measurement
        pulsedmasterlogic.toggle_pulsed_measurement(False)

        # Make sure it stopped (Wait until the Picoscope card sends the data)
        while pulsedmasterlogic.status_dict['measurement_running']:
            time.sleep(0.1)

        #######################################################################################################
        # Analysis
        ######################################################################################################

        fom_func = script_params['fom_function']
        if script_params['fom_func_volt_2_rabi'] is None:
            fom = fom_func()
        else:
            pulse = ArbPulse.load_pulse(folder_path, filename, unit_t='s', unit_data='V')
            fom = fom_func(pulse, func_volt_2_rabi=script_params['fom_func_volt_2_rabi'])
        

        fom_all.append(fom)

        # Update the figure of merit and the standard deviation to the fom logic
        fomlogic.update_fom(fom, script_params['readout_noise'], status_code=0)   # todo: define above
        #fomlogic.update_fom(fom, std, status_code=0)

        # update the last time the fom is calculated
        last_time_fom = time.time()

    # when did the optimization stop?
    opt_end_time = time.time()

    print('It took QuOCS ' + str(opt_end_time-opt_start_time) + ' s to optimize the pulse!')
except:
    logger.exception("")
finally:
    # give a very high fom to the optimization algorithm
    # and status code -1, to interrupt the optimization smoothly
    uglobals.abort.set()
    optimizationlogic.handle_exit_obj.is_user_running = False
    logger.info('Stopping the optimization!')
    fomlogic.update_fom(10**10, status_code=-1)

print("Optimization finished")

In [ ]:
len(pulse['timegrid_ampl'])

## Test FOM noise

In [ ]:
uglobals.abort.clear()

pulsed_settings = dict()
pulsed_settings['invoke_settings'] = True
pulsedmasterlogic.set_measurement_settings(pulsed_settings)
script_params['pulsed_settings'] = pulsed_settings


foms = []
n_mc = 20
idx= 0
try:
    while idx<n_mc and not uglobals.abort.is_set():
        time_stamp=time.time()

        qmeas = cp.deepcopy(script_params['generate_parameters'])
        experiment = qmeas['experiment']
        exp_name = qmeas['name']
        qmeas.pop('experiment', None)
        # generate the sequence
        pulsedmasterlogic.generate_predefined_sequence(experiment,kwarg_dict=qmeas)  

        time.sleep(1)

        # upload the sequence
        pulsedmasterlogic.sample_ensemble(exp_name,True)
        while pulsedmasterlogic.status_dict['sampload_busy'] or pulsedmasterlogic.status_dict['sampling_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:     
            time.sleep(1)

        #print('Finished uploading', dictparams_oc["name"], '!')

        # make sure everything is finished (crucial)
        #time.sleep(1)#time.sleep(5)    

         # start the measurement
        #print('Starting the measurement!')
        pulsedmasterlogic.toggle_pulsed_measurement(True)
        time.sleep(1)

        # make sure the measurement started
        while not pulsedmasterlogic.status_dict['measurement_running']:
            time.sleep(0.1)

        measurement_start_time = time.time()

        while time.time() <= measurement_start_time + runtime:
            time.sleep(0.2)

            # option to stop the measurement
            if uglobals.abort.is_set():
                print('Stopping the measurement!')
                break

        # option to stop the measurement
        if uglobals.abort.is_set():
            print('Stopping the measurement!')
            pulsedmasterlogic.toggle_pulsed_measurement(False)
            break

        #time.sleep(1)

        # Stop the measurement
        pulsedmasterlogic.toggle_pulsed_measurement(False)

        # Make sure it stopped (Wait until the Picoscope card sends the data)
        while pulsedmasterlogic.status_dict['measurement_running']:
            time.sleep(0.1)

        #######################################################################################################
        # Analysis
        ######################################################################################################

        fom_func = script_params['fom_function']
        foms.append(fom_func())

        idx += 1
except:
    logger.exception("")

In [ ]:
idx
foms

In [ ]:
np.mean(foms), np.std(np.asarray(foms))

## Save the results

Currently need to hiz "stop" in optimalcontrol gui manually

In [133]:
try:
    save_optimization_result(folder_path)
except:
    logger.exception("")

In [ ]:
try:
    optimizer_obj = optimizationlogic.optimization_obj
    optimizer_obj.get_optimization_algorithm()
except:
    logger.exception("")

In [ ]:
optimizer_obj.__dict__

In [ ]:
pulses_list, time_grids_list, parameters_list = optimizer_obj.opt_alg_obj.get_best_controls()

t_amplitude = time_grids_list[0]
amplitude = pulses_list[0]
t_phase = time_grids_list[1]
phase = pulses_list[1]

In [ ]:
best_dict = optimizer_obj.opt_alg_obj.get_best_controls()

pulses_list = best_dict['pulses']
time_grids_list = best_dict['timegrids']
parameters_list = best_dict['parameters']

pulses_list

In [ ]:
optimizer_obj.opt_alg_obj.get_best_controls().keys()

## manually extract best pulse from result log

In [ ]:
path = r"C:\Software\qudi_data\optimal_control_assets\20220724_1\20220723_202625_test_dCRAB_Noisy_2_control_fields"
file = r"20220723_202625_best_controls"
pulse = TimeDependentSimulation.load_pulse(path, file, extension='npz')

#path = r"C:\Software\qudi_data\optimal_control_assets\20220724_1"
#file = r"on_nv=2_"
#pulse = load_pulse(path, file, extension='txt')

pulse

In [ ]:
import matplotlib.pyplot as plt
    
t_amplitude = pulse['timegrid_ampl']
amplitude = pulse['data_ampl']
t_phase = pulse['timegrid_phase']
phase = pulse['data_phase']

try:
    plt.plot(t_amplitude*1e9, amplitude, label="Ampl (I)")
    plt.plot(t_phase*1e9, phase, label="Phase (Q)")
    plt.xlabel("time (ns)")
    plt.legend()
    plt.show()
except:
    logger.exception("")

In [ ]:
path = folder_path
name = "opt_pulse_"

data_ampl = np.column_stack((t_amplitude, amplitude))
data_phase = np.column_stack((t_phase, phase))

save_pulse(path, data_ampl, data_phase, name=name)


## Simulation only

In [108]:
script_params['readout_noise'] = 0.01*np.sqrt(2)

script_params['fom_func_volt_2_rabi'] = lambda y: y/ampl_rect*(1e-6/t_rabi_rect)
script_params['fom_function'] = setup_fom_simulation_sigz(f_res)

# debug
#fom_func = script_params['fom_function']
#fom_func(pulse, func_volt_2_rabi=script_params['fom_func_volt_2_rabi'])

script_params['readout_noise']

0.014142135623730952

In [109]:
f_res = 2628.56e6
fom_func = setup_fom_simulation_sigz(f_res)
script_params['fom_func_volt_2_rabi'] = lambda y: y/ampl_rect*(1e-6/t_rabi_rect)




######################################################################################################
# Parameters and Settings
######################################################################################################

# runtime of each experiment
runtime = script_params['runtime_per_epoch']
length_oc = script_params['length_oc'] 

# parameter to stop the experiment if its set to False in the console
pulsedmasterlogic.globalrun = True
uglobals.abort.clear()
logger.info("Starting new closed-loop quocs optimization")


# array to save the fom evolution
fom_all = []

# This section is devoted to the initialization in the pulsed logic and optimization logic of the main
# settings and parameters to be usde in the creation ofthe pulse sequence and the optimization
# Iteration, controls and figure of merit to compare with QuOCS
# Just an example for debug
args_dict = {"is_noisy": script_params['is_noisy']}
qubit = OneQubit2Fields(args_dict)

######################################################################################################
# Measurement
######################################################################################################
optimalcontrol.start_optimization()

# crucial, maybe it can be much shorter
time.sleep(1)

# Just a time to check for latent time
last_time_fom = time.time()
# repeat the whole process until its manually stopped or QuOCS finsihed the optimization
# Wait few seconds before starting to get and return data
while not optimizationlogic.handle_exit_obj.is_user_running:
    time.sleep(0.1)
    if (time.time() - last_time_fom) > 30 or uglobals.abort.is_set():
        logger.warning("Timeout or abort while initializing quocs.")
        uglobals.abort.set()
        break

# iteration number
it_val = 0

# when did the optimization start?
opt_start_time = time.time()
try:
    # print("Check before the loop starts: {0}".format(optimizationlogic.handle_exit_obj.is_user_running))
    while optimizationlogic.handle_exit_obj.is_user_running == True and not uglobals.abort.is_set():
        time_stamp=time.time()
        # wait until QuOCS optimizes the controls
        # print("Wait until the controls logic gives the controls")
        while not controlslogic.are_pulses_calculated:
            time.sleep(0.1)
            # If the waiting time exceed 10 seconds left stop the optimization
            if time.time() - last_time_fom > 20:
                logger.error("Too much time... Exit!")
                optimizationlogic.handle_exit_obj.is_user_running = False
                break

 
        #######################################################################################################
        # Get the Controls
        #######################################################################################################
        # Change the status of control calculations to avoid to evaluate the fom twice with the same controls
        controlslogic.are_pulses_calculated = False
        # Get the controls from the controls logic
        pulses, parameters, timegrids = controlslogic.pulses, controlslogic.parameters, controlslogic.timegrids
        #######################################################################################################
        # Perform the measurement
        #######################################################################################################

        # save the pulses as .txt files (predefined methods doesn't allow us to upload a numpy array as 
        # parameter)
        data_ampl = np.column_stack((timegrids[0], pulses[0]))
        data_phase = np.column_stack((timegrids[0], pulses[1]))
        save_pulse(folder_path, data_ampl, data_phase, name=filename)

        time.sleep(0.5)
        
       
        measurement_start_time = time.time()

        # option to stop the measurement
        if pulsedmasterlogic.globalrun == False or uglobals.abort.is_set():
            print('Stopping the measurement!')
            break


        #######################################################################################################
        # Analysis
        ######################################################################################################

        fom_func = script_params['fom_function']
        if script_params['fom_func_volt_2_rabi'] is None:
            fom = fom_func()
        else:
            pulse = ArbPulse.load_pulse(folder_path, filename, unit_t='s', unit_data='V')
            fom = fom_func(pulse, func_volt_2_rabi=script_params['fom_func_volt_2_rabi'])
        
        fom += np.random.normal(0, script_params['readout_noise'])
        fom = abs(fom)
                                
        fom_all.append(fom)

        # Update the figure of merit and the standard deviation to the fom logic
        fomlogic.update_fom(fom, script_params['readout_noise'], status_code=0)   # todo: define above
        #fomlogic.update_fom(fom, std, status_code=0)

        # update the last time the fom is calculated
        last_time_fom = time.time()

    # when did the optimization stop?
    opt_end_time = time.time()

    print('It took QuOCS ' + str(opt_end_time-opt_start_time) + ' s to optimize the pulse!')
except:
    logger.exception("")
finally:
    # give a very high fom to the optimization algorithm
    # and status code -1, to interrupt the optimization smoothly
    uglobals.abort.set()
    optimizationlogic.handle_exit_obj.is_user_running = False
    logger.info('Stopping the optimization!')
    fomlogic.update_fom(10**10, status_code=-1)

print("Optimization finished")

Stopping the measurement!
It took QuOCS 214.37220668792725 s to optimize the pulse!
Optimization finished
